Code pour projet crypto

Import des librairies

In [2]:
import hashlib
import random 
import gmpy2 as gm
import math
import time
import base64
import os, binascii
from backports.pbkdf2 import pbkdf2_hmac
import pyaes, pbkdf2, secrets


Exercice 1

Partie 1

In [3]:

def trouver_chaine(base, zeros_cibles):
        compteur = 0
        while True:
            chaine_test = base + str(compteur)
            hash_resultat = sha256_hash(chaine_test)
            if hash_resultat.endswith('0' * zeros_cibles):
                return chaine_test, compteur, hash_resultat
            compteur += 1

def sha256_hash(chaine):
        return hashlib.sha256(chaine.encode()).hexdigest()
        
def trouver_chaine_et_stats(base, zeros_cibles, iterations_stat=5):
    

    # Première partie : Trouver la chaîne
    chaine_trouvee, compteur_trouve, hash_trouve = trouver_chaine(base, zeros_cibles)

    # Deuxième partie : Statistiques de temps
    temps_n = []
    temps_n_plus_1 = []
    
    for _ in range(iterations_stat):
        debut = time.time()
        trouver_chaine(base, zeros_cibles)
        fin = time.time()
        temps_n.append(fin - debut)

        debut = time.time()
        trouver_chaine(base, zeros_cibles + 1)
        fin = time.time()
        temps_n_plus_1.append(fin - debut)

    temps_moyen_n = sum(temps_n) / len(temps_n)
    temps_moyen_n_plus_1 = sum(temps_n_plus_1) / len(temps_n_plus_1)

    rapport_temps = temps_moyen_n_plus_1 / temps_moyen_n

    return {
        "chaine_trouvee": chaine_trouvee,
        "hash_trouve": hash_trouve,
        "compteur_trouve": compteur_trouve,
        "temps_moyen_n": temps_moyen_n,
        "temps_moyen_n_plus_1": temps_moyen_n_plus_1,
        "rapport_temps": rapport_temps
    }


base_chaine = "Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6"
zeros_cibles = 4 # Nombre de zéros à la fin du hash
resultats = trouver_chaine_et_stats(base_chaine, zeros_cibles)
resultats


{'chaine_trouvee': 'Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA677031',
 'hash_trouve': '31e6f8e6377ad1c9895ef7c22b0ef9a252c0f0ec9d062178c58b416bcb470000',
 'compteur_trouve': 77031,
 'temps_moyen_n': 0.1148106575012207,
 'temps_moyen_n_plus_1': 0.32151641845703127,
 'rapport_temps': 2.8004056892855336}

Exercice 2

In [10]:
with open("format .txt", "r",encoding="utf-8") as file:
    text = str(file.read())
text

"Le marché des sneakers en France a connu une croissance exponentielle au cours des dernières années, devenant un phénomène de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumière ses dynamiques économiques, culturelles et sociales.\nD'un point de vue économique, le commerce de sneakers en France est stimulé par une demande croissante pour des modèles rares et exclusifs. \nLes éditions limitées et les collaborations entre marques de renom et designers célèbres créent une urgence d'achat, poussant les prix sur le marché secondaire à atteindre des sommets vertigineux. \nCette situation a engendré l'émergence d'une véritable économie parallèle, où les sneakers sont non seulement des objets de mode, mais également des actifs spéculatifs.\n\nSur le plan culturel, les sneakers occupent une place prépondérante dans l'univers de la mode et de la 

In [42]:
password = "123456789"
passwordSalt = os.urandom(16)
key = pbkdf2.PBKDF2(password, passwordSalt).read(32)
iv = secrets.randbits(256)
#On est obliger de recrée un mot de pass unique pour chiffrer le texte
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
ciphertext = aes.encrypt(text)

In [43]:
print(aes.decrypt(ciphertext))

b'\xedoW\xb9\x8bf\xcf\xb8\x96\xcb\xf7U\xb2\xd0h;\xd8v?\x06Tk)\xceMz\xb1\x8a\xd3y\x873\xe2\xf9%J{\xc4\xa0b\x90\xf6\xf7dZ!C\x9b\xa3cD\xc5\x02\xdc\xae\x1f0\x964\xb3\xb3\x92X*\xce\xfbH\xbb\x1f1SZ_\xf1\xc9\xfe\xcd\xd1\xde\x05\x96\x90\xbcgN\x0e\x89\x01\xba\xe9\x85\x1aow\xf9\x17\xe2\xdc\xaau\n\xab\xc9\x91\xb2\x1b\xc7\\\xe0`U\x0cC\nT\x08\xc5s\x16\xb8\xb6\x1d\xf4\tY\xab\x8e{5k\xd6\x9f\xf6<\xca\xbf\xa0\x95\xf8\xe6o\x18\xbe\xf7jF\xc5\xc4\xd1\x04\x81\xb1\x17\xae\xb2\x0bx\xab&p\xdd\x15\x94U\x12+\xf2Cy\x05\xd1\xfc\xa3Tg\xfaZj\x85crD\xceN,\xd5\x94\x87\x92*\x9e\x8d3\x145UG\x8c\xb3\x882%a(\xb0P\x1c\xac\xc5\xc4B\xc9\x0bs\xb6\x1c\xcf\xc3\xfe\x84\xb2\x1b\xcd\x16f\xfa\x82\xca\x0eC\xecc`\xac, :\xa1(\xb0\xc6k\xb90&\xcd\xa5z6\xe1\xcf\xdd\xa9\x95\xe2\xf5\xdb?b\x96\xe4Z\x81\xab\xfdz4\x92\xa9R6q\'\x80\xa4g\x83\x87\xe8W\xf4\xf1~\xf0\xbc:\x1f@&\x00\xf71\xb6$\xfed\xc3W(1H\n\x9e\xf3\xea\xc8eT\xdf\xf9\x05\t\xfb\xd8\x84u\x8cl\x8b\x9b\xaa\x1d\xf4\xfatw`\x96\xe0\t\x1a~\xd2h\'=\x8bSw\x93\xf7\x7f\xda\x0f\x91\x04\xf2P{\xcb

In [50]:
print(pyaes.Counter(iv))
#Et du coup de le regénéré
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
print(aes.decrypt(ciphertext))

b"Le march\xe9 des sneakers en France a connu une croissance exponentielle au cours des derni\xe8res ann\xe9es, devenant un ph\xe9nom\xe8ne de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumi\xe8re ses dynamiques \xe9conomiques, culturelles et sociales.\nD'un point de vue \xe9conomique, le commerce de sneakers en France est stimul\xe9 par une demande croissante pour des mod\xe8les rares et exclusifs. \nLes \xe9ditions limit\xe9es et les collaborations entre marques de renom et designers c\xe9l\xe8bres cr\xe9ent une urgence d'achat, poussant les prix sur le march\xe9 secondaire \xe0 atteindre des sommets vertigineux. \nCette situation a engendr\xe9 l'\xe9mergence d'une v\xe9ritable \xe9conomie parall\xe8le, o\xf9 les sneakers sont non seulement des objets de mode, mais \xe9galement des actifs sp\xe9culatifs.\n\nSur le plan culturel, les s

In [55]:
p = 7946851324679854613245823
g = 5
A = 7579501795988122393422986
## Elle publie A
password = "35426517"
IV = "756431" 
N = int(password + "0000" + IV) 

b = 123456789  

# Calcul de B et C pour ElGamal
B = pow(g, b, p)
K = pow(A, b, p)  # Clé secrète partagée
C = (K * N) % p  # Chiffrement de N avec la clé secrète partagée

B, C

(387225946877039798412155, 5883019579684883513940408)

Exercice 3